# Sub-Guaussianについて

参考：
* [Bandit Algorithms](https://tor-lattimore.com/downloads/book/book.pdf)

## 裾確率

**モチベーション**

まずは簡単なケースとして，次を考えます．

* $X, X_1, X_2, \ldots, X_n$をi.i.d.な確率変数とします
* $\mu=\mathbb{E}[X]$と$\sigma^2=\mathbb{V}[X]$は存在するとします．
* $\hat{\mu}=\frac{1}{n} \sum_{i=1}^n X_i$によって期待値を推定したいとします．

このとき，この推定の精度を分散で測ってみると，

$$\mathbb{V}[\hat{\mu}]=\mathbb{E}\left[(\hat{\mu}-\mu)^2\right]=\frac{\sigma^2}{n}$$

なので，直感的には$n$に対して線形に$\mu$と$\hat{\mu}$の差が縮んでいくことがわかります．
一方で，これは誤差の分布の形状については何も言っていません．そこで，次のような確率について考えてみます：

$\mathbb{P}(\hat{\mu} \geq \mu+\varepsilon) \quad$ and $\quad \mathbb{P}(\hat{\mu} \leq \mu-\varepsilon)$

これを$\hat{\mu}-\mu$の裾確率と呼びます．
分散を考える代わりに，この裾確率を上からバウンドすることで，誤差の分布の形状について議論したいわけですね．


## マルコフとチェビシェフの不等式

裾確率を上からバウンドするための最も基本的なツールがマルコフの不等式とチェビシェフの不等式です．

---

**補題**

任意の確率変数$X$と$\varepsilon > 0$について，次が成立する：

* (a) (Markov): $\mathbb{P}(|X| \geq \varepsilon) \leq \frac{\mathbb{E}[|X|]}{\varepsilon}$.
* (b) (Chebyshev): $\mathbb{P}(|X-\mathbb{E}[X]| \geq \varepsilon) \leq \frac{\mathbb{V}[X]}{\varepsilon^2}$.

マルコフの不等式はチェビシェフからすぐわかります．

---

チェビシェフの不等式を使えば，両側裾確率は

$$
\mathbb{P}(|\hat{\mu}-\mu| \geq \varepsilon) \leq \frac{\sigma^2}{n \varepsilon^2}
$$

として抑えることができます．

これは期待値と分散が存在すればいつでも使えるので便利ですが，$X$に高次のモーメントが存在する場合は荒いバウンドになってしまいます．この場合はマルコフの不等式を$|\hat{\mu}-\mu|^k$に対して適用すればいいですが，Cramer-Chernoff methodを使うと，$k$が連続の一般的な場合について考えることができます．

### 集中極限定理

Cramer-Chernoff methodがChebyshevより嬉しいポイントをわかりやすくするために，集中極限定理について考えてみましょう．

* $S_n=\sum_{t=1}^n\left(X_t-\mu\right)$

とします．分散が存在するとき，集中極限定理は，
$n \rightarrow \infty$における$S_n / \sqrt{n \sigma^2}$ の分布が期待値０で分散１のガウス分布に飛ぶことを示しています．
集中極限定理を使って，簡単に裾確率のバウンドを考えてみましょう．

$Z \sim \mathcal{N}(0,1)$を考えます．定義から，

$$
\mathbb{P}(Z \geq u)=\int_u^{\infty} \frac{1}{\sqrt{2 \pi}} \exp \left(-\frac{x^2}{2}\right) d x
$$

です．これを上からバウンドしてみます：

$$
\begin{aligned}
\int_u^{\infty} \frac{1}{\sqrt{2 \pi}} \exp \left(-\frac{x^2}{2}\right) d x & \leq \frac{1}{u \sqrt{2 \pi}} \int_u^{\infty} x \exp \left(-\frac{x^2}{2}\right) d x \\
& =\sqrt{\frac{1}{2 \pi u^2}} \exp \left(-\frac{u^2}{2}\right)
\end{aligned}
$$

これを抑えたかった裾確率に適用してみましょう（これは$n\geq 30$くらいからしか良い菌時にはなりません）．

$$
\begin{aligned}
\mathbb{P}(\hat{\mu} \geq \mu+\varepsilon) & =\mathbb{P}\left(S_n / \sqrt{\sigma^2 n} \geq \varepsilon \sqrt{n / \sigma^2}\right) \approx \mathbb{P}\left(Z \geq \varepsilon \sqrt{n / \sigma^2}\right) \\
& \leq \sqrt{\frac{\sigma^2}{2 \pi n \varepsilon^2}} \exp \left(-\frac{n \varepsilon^2}{2 \sigma^2}\right) .
\end{aligned}
$$

一方で，チェビシェフの不等式では

$$
\mathbb{P}(|\hat{\mu}-\mu| \geq \varepsilon) \leq \frac{\sigma^2}{n \varepsilon^2}
$$

として抑えていました．明らかに集中極限定理で求めたバウンドのほうが早く収束します．
Cramer-Chernoff methodでは，このテクニックを有限時間で使えるようにしたものです．


### Cramer-Chernoff methodとSubgaussian 

Cramer-Chernoffの話は，次のSubgaussianを満たす確率変数に対して威力を発揮します．

---

**定義：Sub-gaussian**

確率変数$X$は次の条件を満たすときに$\sigma$-subgaussian と呼びます．任意の$\lambda \in \mathbb{R}$について，

$$
\mathbb{E}[\exp (\lambda X - \sigma^2 / 2)] \leq \exp \left(\lambda^2\right)
$$

---

これはモーメント母関数を使っても定義することができます．モーメント母関数を$M_X(\lambda)=\mathbb{E}[\exp (\lambda X)]$とすると，上の定義は

$$
\psi_X(\lambda)=\log M_X(\lambda) \leq \frac{1}{2} \lambda^2 \sigma^2 \quad \text { for all } \lambda \in \mathbb{R}
$$

と同じです．ここで，$\psi_X$はキュムラント母関数と呼ばれます．

---

**モーメント母関数の性質**

モーメント母関数$M_X(\lambda)$は$\lambda$で$n$回微分して$\lambda=0$を代入すると$\mathbb{E}[X^n]$になります．そもそも，ある数列

$$a_1, a_2, \ldots, a_n, \ldots$$

がある関数$g(\lambda)$のべき級数展開の係数である（つまり$g(\lambda)=\sum_{n=1}^{\infty} a_n \lambda^n$）のとき，$g(\lambda)$は数列$\{a_n\}$に対する母関数であるといいます．
モーメント母関数は

$$
M_X(\lambda)=\sum_{m=0}^{\infty} \frac{E\left[X^m\right]}{m !} \lambda^m
$$

と同じなので，母関数なのです．

$X$が任意の$\lambda > 0$について$M_X(\lambda)=\infty$の場合，確率変数$X$は**裾が重い**といいます．例えば

$$\mathbb{E}[\exp (\lambda X)]=\int_0^{\infty} {\exp (-\sqrt{x})} \times \exp (\lambda x) d x=\infty$$

です．

---

**キュムラント母関数の性質**

独立な確率変数$X$と$Y$の和のキュムラント母関数は，それぞれの確率変数のキュムラント母関数の和に等しくなります．これを使うことで確率変数の和についての挙動を解析することが簡単になります．

---

さて，Cramer-Chernoff methodを使って，なぜ「Subgaussian」という名前なのか見てみましょう．
$\sigma$-subgaussianな確率変数の裾は，分散が同じガウス分布の裾とだいたい同じ速度で落ちていきます：

---

**定理**

もし$X$が$\sigma$-subgaussianなら，任意の$\varepsilon \geq 0$に対して，

$$
\mathbb{P}(X \geq \varepsilon) \leq \exp \left(-\frac{\varepsilon^2}{2 \sigma^2}\right)
$$

が成立する．

**証明**

これを証明するためにCramer-Chernoff methodを使います．
$\lambda > 0$を後で設定するパラメータとします．このとき，

$$
\begin{aligned}
\mathbb{P}(X \geq \varepsilon) & =\mathbb{P}(\exp (\lambda X) \geq \exp (\lambda \varepsilon)) \\
& \leq \mathbb{E}[\exp (\lambda X)] \exp (-\lambda \varepsilon) \\
& \leq \exp \left(\frac{\lambda^2 \sigma^2}{2}-\lambda \varepsilon\right)
\end{aligned}
$$

が成立します．一個目の不等式はマルコフの不等式，２つ目の不等式はSubgaussianの定義です．
あとは$\lambda = \varepsilon / \sigma^2$とすれば終わりです．

---

ガウス分布と比較してみましょう．$Z \sim \mathcal{N}(0,\sigma^2)$とすると，

$$
\mathbb{P}(Z \geq u)\lesssim \exp \left(-\frac{u^2}{2\sigma^2}\right)
$$

なので，落ちる速度はだいたい同じです．
Subgaussianには次の性質があります：

---

**Sub-gaussianの性質**

1. $\mathbb{E}[X]=0$ and $\mathbb{V}[X] \leq \sigma^2$
2. $c\in \mathbb{R}$について，$cX$は$|c|\sigma$-subgaussian
3. $X_1 + X_2$は$\sqrt{\sigma^2_1 + \sigma_2^2}$-subgaussian

---

これを使うと，特に$X_i-\mu$が独立で$\sigma$-subgaussianの場合に簡単に裾確率をバウンドすることができます．
実際，

$$
\mathbb{P}(\hat{\mu} \geq \mu+\varepsilon) \leq \exp \left(-\frac{n \varepsilon^2}{2 \sigma^2}\right) \quad \text { and } \quad \mathbb{P}(\hat{\mu} \leq \mu-\varepsilon) \leq \exp \left(-\frac{n \varepsilon^2}{2 \sigma^2}\right)
$$

が成立します．